In [1]:
import pandas as pd
import numpy as np
import math

In [5]:
import copy

## Load data

In [7]:
active_location = pd.read_excel('Active Locations.xlsx')
#parking_garage = pd.read_csv('parking_garage_list.csv', delimiter='\t').drop(['Unnamed: 0'], axis=1)
parking_garage = pd.read_csv('parking_garage_list.csv', delimiter='\t')

In [ ]:
parking_garage.head(1)

In [58]:
parking_garage.shape

(1791, 17)

In [59]:
active_location.shape

(838, 12)

## Clean data

In [46]:
# Drop NaN column in active_location
active_location = active_location[~active_location['Address 1'].isna()]

In [47]:
parking_garage['Address'] = parking_garage[['Address Building', 'Address Street Name']].apply(lambda x: x[0] + ' ' + x[1], axis=1)
parking_garage['Address'].apply(lambda x: x.split(' ')[-1]).value_counts()

ST            996
AVE           490
BLVD           75
BROADWAY       37
PL             22
W              19
AMERICAS       18
S              16
RD             14
PLZ            14
TER            11
STREET         10
CONCOURSE      10
C               9
PKWY            7
DR              7
AVENUE          4
LN              3
SQ              3
EXT             3
CIR             2
ROW             2
BOWERY          2
BOULEVARD       2
820             1
TPKE            1
HWY             1
PLACE           1
Street          1
CITY            1
Avenue          1
N               1
EXPY            1
HTS             1
SLIP            1
E               1
EXPRESSWAY      1
H               1
PLAZA           1
Name: Address, dtype: int64

#### Create common street abbreviations

In [48]:
street_abbreviation_mapping = { 
    'aly' : 'Alley',
    'ave' : 'Avenue',
    'blv' : 'Boulevard',
    'blvd' : 'Boulevard',
    'blvd.' : 'Boulevard',
    'cir' : 'Circle',
    'ct' : 'Court',
    'cv' : 'Cove',
    'cyn' : 'Canyon',
    'dr' : 'Drive',
    'expy' : 'Expressway',
    'hwy' : 'Highway',
    'ln' : 'Lane',
    'pkwy' : 'Parkway',
    'pkwy.' : 'Parkway',
    'parkwy' : 'Parkway',
    'pl' : 'Place',
    'pt' : 'Point',
    'rd' : 'Road',
    'sq' : 'Square',
    'st' : 'Street',
    'ter' : 'Terrace',
    'tr' : 'Trail',
    'trl' : 'Trail',
    'wy' : 'Way'
}

direction_abbreviation_mapping = {
    'w': 'West',
    'e': 'East',
    's': 'South',
    'n': 'North'
}

street_abbreviation_mapping = {' ' + k.upper(): ' ' + v.upper() for (k, v) in street_abbreviation_mapping.items()}
direction_abbreviation_mapping = {' ' + k.upper() + ' ':' ' + v.upper() + ' ' for (k, v) in direction_abbreviation_mapping.items()}

In [49]:
def replace_abbreviation(address, street_mapping, direction_mapping):
    new_address = address
    for k, v in direction_mapping.items():
        new_address = new_address.replace(k, v, 1)
    count = 0
    for k, v in street_mapping.items():
        if count >= 1:
            break
        tmp = new_address.replace(k, v, 1)
        if tmp != new_address:
            new_address = tmp
            count += 1
    return new_address

In [50]:
parking_garage['Address'] = parking_garage['Address'].apply(lambda s: replace_abbreviation(s, street_abbreviation_mapping, direction_abbreviation_mapping))
parking_garage['Address']

0            219 EAST 69TH STREET
1              2 NORTH 6TH STREET
2            219 EAST 69TH STREET
3                74 DEKALB AVENUE
4        220 RIVERSIDE BOULEVARDD
                  ...            
1786         201 WEST 75TH STREET
1787       9525 QUEENS BOULEVARDD
1788    3720 HENRY HUDSON PARKWAY
1789         55B RICHMOND TERRACE
1790             800 BRADY AVENUE
Name: Address, Length: 1791, dtype: object

In [51]:
# now lower case addresses in both datasets
parking_garage['Address'] = parking_garage['Address'].apply(lambda s: s.lower())
active_location['Address'] = active_location['Address 1'].apply(lambda s: s.lower())

# parking_garage['zipcode'] = parking_garage['Address ZIP']
# active_location['zipcode'] = active_location['ZIP Code']

In [52]:
parking_garage.head(1)

,Unnamed: 0,DCA License Number,License Type,License Expiration Date,Business Name,Business Name 2,Address Building,Address Street Name,Secondary Address Street Name,Address City,Address State,Address ZIP,Contact Phone Number,Address Borough,Borough Code,Address,zipcode
0,NaN,1306493-DCA,Business,3/31/21,69 ENTERPRISE PARKING LLC,NaN,219,E 69TH ST,NaN,NEW YORK,NY,10021.0,2126270012,Manhattan,1.0,219 east 69th street,10021.0


In [53]:
active_location.head(1)

,Location,Name,Phone 1,Address 1,City,State,ZIP Code,Software,Zone,OS Version,Address,zipcode
0,ABC000,Main Office,212-736-7171,107 West 13th Street,New York,New York,10011,NaN,Not a garage,NaN,107 west 13th street,10011


In [62]:
# pd.merge(parking_garage, active_location, on=['Address'])[['Business Name', 'Address Building', 'Address Street Name', 'Address', 'Address ZIP', 
#                                                            'Name', 'Address 1', 'ZIP Code']]

# https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
joined = pd.merge(parking_garage, active_location, on=['Address'])


In [64]:
parking_garage['label'] = parking_garage['DCA License Number'].isin(joined['DCA License Number'])

In [68]:
parking_garage.to_csv('./parking_garage_labeled.csv')

In [57]:
parking_garage['Address']

0            219 east 69th street
1              2 north 6th street
2            219 east 69th street
3                74 dekalb avenue
4        220 riverside boulevardd
                  ...            
1786         201 west 75th street
1787       9525 queens boulevardd
1788    3720 henry hudson parkway
1789         55b richmond terrace
1790             800 brady avenue
Name: Address, Length: 1791, dtype: object

In [58]:
active_location['Address 1']

0        107 West 13th Street
1        100 West 89th Street
2        520 West 43rd Street
3             59 Allen Street
4      2600 Netherland Avenue
                ...          
837           101 Arch Street
838    2155 East Tremont Ave.
839            800 5th Avenue
840      450 East 83rd Street
841          80 DeKalb Avenue
Name: Address 1, Length: 842, dtype: object

In [59]:
type(active_location)

pandas.core.frame.DataFrame

In [60]:
active_location.to_csv("active_location.csv")

PermissionError: [Errno 13] Permission denied: 'active_location.csv'

#### For loop to build zipcode level dataframes of parking_garage and  active_location 

#### Left merge all parking_garage and active_location zip code files on address
- Do this twice: once including all matches and once with non-matches
- For all active_location zip codes with no matches, dump in another dataset

#### Combine matched and unmatched into single dataset

In [61]:
parking_garage.describe()

,Address ZIP,Borough Code
count,1790.000000,1756.000000
mean,10351.874302,1.675399
std,562.108860,1.049482
min,0.000000,1.000000
25%,10019.000000,1.000000
50%,10034.000000,1.000000
75%,10459.000000,2.000000
max,11436.000000,5.000000


In [62]:
parking_garage['Address ZIP'].unique()

array([10021., 11249., 11201., 10069., 11432., 11377., 10040., 10003.,
       10012., 10011., 10001., 10065., 10032., 10005., 11206., 11101.,
       10019., 10016., 11215., 11235., 11354., 10006., 10009., 11217.,
       11385., 10128., 10036., 10033., 10038., 10075., 11373., 10025.,
       11211., 10457., 10452., 10453., 10463., 10458., 10013., 11238.,
       10014., 11375., 11435., 10023., 11360., 10022., 10451., 11205.,
       10028., 11209., 10018., 10024., 10475., 11372., 10004., 11368.,
       11105., 10035., 10017., 11374., 10026., 10456., 10461., 11222.,
       10010., 10007., 11231., 10459., 10460., 10468., 10282., 10020.,
       11102., 10280., 11208., 10002.,    nan, 10304., 10034., 11224.,
       10462., 10455., 10037., 11430., 10027., 10031., 11216., 10029.,
       11418., 11226., 10166., 11040., 11433., 11109., 10454., 10152.,
       11369., 11355., 11220., 11416., 11225., 11423., 11212., 11417.,
       11218., 11434., 10030., 11234., 11106., 10153., 11207., 11221.,
      

In [70]:
# https://datatofish.com/integers-to-strings-dataframe/

parking_garage['Address ZIP'] = parking_garage['Address ZIP'].apply(str).str[:5]

In [71]:
active_location.describe()

,Location,Name,Phone 1,Address 1,City,State,ZIP Code,Software,Zone,OS Version
count,842,842,737,838,837,837,836,827,842,98
unique,842,790,725,828,30,8,123,4,17,2
top,INT343,SP Plus Corporation,617-451-1393,107 West 13th Street,New York,New York,10021,Ultra,East of Central Park,Windows 7
freq,1,19,6,3,602,756,59,762,147,70


In [72]:
active_location['ZIP Code'].unique()

array(['10011', '10024', '10036', '10002', '10463', '10029', '10016',
       '10028', '10017', '10065', '07024', '10022', '11374', '10003',
       '10039', '10128', '10019', '10026', '10038', '10018', '10021',
       '10014', '10068', '11211', '10001', '10075', '10040', '10451',
       '10035', '10005', '10471', '11249', '10013', '10025', '11217',
       '10032', '11248', '11101', '11201', '11435', '10301', '11372',
       '19107', '10462', '10452', '20005', '20036', '20009', '20007',
       '20004', '20037', '20006', '20002', '11375', '10007', '10012',
       '10020', '07102', '11215-5606', '11207', '11415', '11216', '11355',
       '10034', '10166', '10027', '11222', '07030', '07306', '10023',
       '08102', '10468', '11225', '11215', '11210', '11238', '10458',
       '11432', '10459', '10162', nan, '10010', '10280', '11209', '10037',
       '10031', '10454', '10069', '10033', '11205', '10009', '20001',
       '11418', '11206', '11373', '02109', '02210', '10004', '10006',
       '10

In [75]:
# https://stackoverflow.com/questions/40973687/create-new-dataframe-in-pandas-with-dynamic-names-also-add-new-column
# https://docs.python.org/2/library/copy.html

parking_garage_dict = {}

for zip_code in parking_garage['Address ZIP'].unique():
    key_name = 'parking_garage'+str(zip_code)
    parking_garage_dict[key_name] = copy.deepcopy(parking_garage)
    to_change = parking_garage['Address ZIP']= zip_code
    parking_garage_dict[key_name].loc[to_change, 'new_col'] = zip_code
    
parking_garage_dict.keys()

dict_keys(['parking_garage10301'])

In [76]:
parking_garage_dict{'parking_garage11368.0'}

SyntaxError: invalid syntax (<ipython-input-76-56ec323c89bc>, line 1)

In [77]:
parking_garage_dict

{'parking_garage10301':       DCA License Number License Type License Expiration Date  \
 0            1306493-DCA     Business                 3/31/21   
 1            2068286-DCA     Business                 3/31/21   
 2            2093168-DCA     Business                 3/31/21   
 3            2093213-DCA     Business                 3/31/21   
 4            1217500-DCA     Business                12/22/19   
 ...                  ...          ...                     ...   
 1787         0945081-DCA     Business                 3/31/11   
 1788         1023251-DCA     Business                 3/31/21   
 1789         2086322-DCA     Business                 7/31/19   
 1790         1397433-DCA     Business                  7/6/18   
 10301                NaN          NaN                     NaN   
 
                                  Business Name Business Name 2  \
 0                    69 ENTERPRISE PARKING LLC             NaN   
 1                     2 NORTH 6TH PARKING CORP  

In [78]:
# https://www.datacamp.com/community/tutorials/python-dictionary-tutorial#dictionary_operations

#parking_garage_dict.get('parking_garage10020.0')
parking_garage_dict.get('parking_garage10020.0').to_csv('parking_garage10020.csv')

AttributeError: 'NoneType' object has no attribute 'to_csv'

In [79]:
active_location_dict = {}

for zip_code in active_location['ZIP Code'].unique():
    key_name = 'active_location'+str(zip_code)
    active_location_dict[key_name] = copy.deepcopy(active_location)
    to_change = active_location['ZIP Code']= zip_code
    active_location_dict[key_name].loc[to_change, 'new_col'] = zip_code
    
active_location_dict.keys()

dict_keys(['active_location10011', 'active_location10024', 'active_location10036', 'active_location10002', 'active_location10463', 'active_location10029', 'active_location10016', 'active_location10028', 'active_location10017', 'active_location10065', 'active_location07024', 'active_location10022', 'active_location11374', 'active_location10003', 'active_location10039', 'active_location10128', 'active_location10019', 'active_location10026', 'active_location10038', 'active_location10018', 'active_location10021', 'active_location10014', 'active_location10068', 'active_location11211', 'active_location10001', 'active_location10075', 'active_location10040', 'active_location10451', 'active_location10035', 'active_location10005', 'active_location10471', 'active_location11249', 'active_location10013', 'active_location10025', 'active_location11217', 'active_location10032', 'active_location11248', 'active_location11101', 'active_location11201', 'active_location11435', 'active_location10301', 'acti

In [80]:
active_location_dict.get('active_location10166').to_csv('active_location10166.csv')

PermissionError: [Errno 13] Permission denied: 'active_location10166.csv'